In [1]:
import math
import numpy as np

In [2]:
sig_digits = 4
#eq_1 = np.random.uniform(low=0, high=13.3, size=(2,)).tolist()
#eq_2 =  np.random.uniform(low=0, high=13.3, size=(2,)).tolist()
#A = np.array([eq_1, eq_2])
#b = np.random.uniform(low=0.5, high=13.3, size=(2,)).tolist()
#b = np.array(b)
#b = b.T

A =np.array([[4.03,2.16],[6.21,3.35]])
b=np.array([-4.61,-7.19])
b=b.T


In [3]:
A[0]

array([4.03, 2.16])

In [4]:
b

array([-4.61, -7.19])

In [5]:
def Cal_LU(D,g, r,nr, sd):
    A=np.array((D),dtype=float)
    f=np.array((g),dtype=float)
    n = f.size
    for i in range(0,n-1):     # Loop through the columns of the matrix
        for j in range(i+1,n):     # Loop through rows below diagonal for each column
            if A[i,i] == 0:
                print("Error: Zero on diagonal!")
                print("Need algorithm with pivoting")
                break
            m = A[j,i]/A[i,i]
            m=r(m,sd) # perform round off
            A[j,:] = A[j,:] - m*A[i,:]
            A[j,:] = nr(A[j,:],sd)
            f[j] = f[j] - m*f[i]
            f[j] = r(f[j],sd)
    return A,f


def Cal_LU_pivot(D,g, r,nr,sd):
    A=np.array((D),dtype=float)
    f=np.array((g),dtype=float)
    n = len(f)
    for i in range(0,n-1):     # Loop through the columns of the matrix
        
        if np.abs(A[i,i])==0:
            for k in range(i+1,n):
                if np.abs(A[k,i])>np.abs(A[i,i]):
                    A[[i,k]]=A[[k,i]]             # Swaps ith and kth rows to each other
                    f[[i,k]]=f[[k,i]]
                    break
                    
        for j in range(i+1,n):     # Loop through rows below diagonal for each column
            m = A[j,i]/A[i,i]
            m = r(m,sd) # perform roundoff
            A[j,:] = A[j,:] - m*A[i,:]
            A[j,:] = nr(A[j,:],sd)
            f[j] = f[j] - m*f[i]
            f[j] = r(f[j], sd)
    return A,f


def Back_Subs(A,f,r, sd):
    n = f.size
    x = np.zeros(n)             # Initialize the solution vector, x, to zero
    x[n-1] = f[n-1]/A[n-1,n-1]    # Solve for last entry first
    x[n-1] = r(x[n-1],sd)
    for i in range(n-2,-1,-1):      # Loop from the end to the beginning
        sum_ = 0
        for j in range(i+1,n):        # For known x values, sum and move to rhs
            sum_ = sum_ + A[i,j]*x[j]
        x[i] = (f[i] - sum_)/A[i,i]
        x[i] = r(x[i],sd)
    return x

In [6]:
def np_roundoff(numbers , significant_digits):
    i=0
    while(i < len(numbers)):
        numbers[i] = numbers[i] + (5 * (10**(-(significant_digits+1))))
        stepper = 10.0 ** significant_digits
        numbers[i] = math.trunc(stepper * numbers[i]) / stepper
        i =i+1
    return numbers

In [7]:
def roundoff_(number, significant_digits):
    number = number + (5 * (10**(-(significant_digits+1))))
    stepper = 10.0 ** significant_digits
    number = math.trunc(stepper * number) / stepper
    return number

In [8]:
def roundoff(number, significant_digits):
    d, n = math.modf(number)
    count=0
    num = n
    while(num > 0):
        num = num // 10
        count = count + 1
    if n == 0:
        significant_digits = significant_digits-1
    d = round(d, significant_digits-count)
    return n+d

In [9]:
def ge_with_pivot(A, b, r=roundoff, sd=2):
    A, f = Cal_LU_pivot(A, b, r=r,nr=np_roundoff,sd=sd)
    result = np_roundoff(Back_Subs(A,f),sd)
    return result

In [10]:
def ge_without_pivot(A,b, r=roundoff,sd=2):
    A,f = Cal_LU(A,b, r=r,nr=np_roundoff,sd=sd)
    result = np_roundoff(Back_Subs(A,f),sd)
    return result

In [28]:
ge_without_pivot(A,b,r=roundoff,sd=4)

array([ 1.01  , -4.0186])

In [30]:
ge_with_pivot(A,b,r=roundoff,sd=4)

array([ 1.01  , -4.0186])